In [ ]:
# Set up tools

In [2]:

import requests

In [3]:

class Tool:
    def __init__(self, name, func, description):
        self.name = name
        self.func = func
        self.description = description

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

class writefiletool:
    def __call__(self, content, file_path):
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        return f"Data successfully written to file: {file_path}"

class ReadFileTool:
    def __call__(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content

class MySerpAPIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        self.api_url = "https://serpapi.com/search"

    def run(self, query):
        params = {
            "q": query,
            "api_key": self.api_key,
        }

        response = requests.get(self.api_url, params=params)

        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.status_code} - {response.text}"

api_key = "Enter your Api" 
my_search = MySerpAPIWrapper(api_key)
query = "Python language"

tools = [
    Tool(
        name="search",
        func=my_search.run,  
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    writefiletool(),
    ReadFileTool(),
]

for tool in tools:
    if hasattr(tool, 'func'):  
        output = tool(query)  
        print(f"{tool.name} Output: {output}")
    elif isinstance(tool, (writefiletool, ReadFileTool)):  
        if isinstance(tool, writefiletool):
            output = tool("Sample content", "sample_file.txt")
        elif isinstance(tool, ReadFileTool):
            output = tool("sample_file.txt")
        print(f"{tool.__class__.__name__} Output: {output}")


search Output: Error: 401 - {
  "error": "Invalid API key. Your API key should be here: https://serpapi.com/manage-api-key"
}
writefiletool Output: Data successfully written to file: sample_file.txt
ReadFileTool Output: Sample content


In [ ]:
# Set up memory

In [14]:

class InMemoryDocstore:
    def __init__(self):
        self.documents = {}

    def store(self, document):
        try:
            self.documents[document["id"]] = document
            return self.documents
        except Exception as e:
            print(e)

    def retrieve(self, id):
        try:
           return self.documents.get(id)
        except Exception as e:
            print(e)

    def update(self, id, updated_document):
      try:
        if id in self.documents:
            self.documents[id].update(updated_document)
        else:
            print(f"Document with ID {id} not found.")
      except Exception as e:
           print(e)


docstore = InMemoryDocstore()

print(docstore.store({"id": "1", "text": "This is a document."}))

doc = docstore.retrieve("1")
print(doc)

docstore.update("1", {"text": "This is an updated document."})

doc = docstore.retrieve("1")
print(doc)



{'1': {'id': '1', 'text': 'This is a document.'}}
{'id': '1', 'text': 'This is a document.'}
{'id': '1', 'text': 'This is an updated document.'}


In [ ]:
# Setup model and AutoGPT

In [26]:
import openai

openai.api_key = "Enter your openai api key"

def embed_text(text):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=text,
        max_tokens=50,  
        temperature=0,
    )
    
    try:
        embedding = response["choices"][0]["text"]
        return embedding
    except (KeyError, IndexError) as e:
        print(f"Error: Unable to extract embedding from API response. {e}")
        return None

def embed_documents(documents):
    embeddings = []
    for document in documents:
        embeddings.append(embed_text(document))
    return embeddings

text = "write a weather report for SF today"
embeddings = embed_text(text)
print(embeddings)




Today in San Francisco, we are expecting a high of 64 degrees with a low of 54 degrees. The skies will be mostly cloudy with a chance of light rain in the afternoon. Winds will be light and variable. It will be a great


In [17]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [18]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(3) 
embeddings = np.array([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
])

assert embeddings.shape[1] == index.d, "Dimensionality of embeddings does not match index dimensionality"
index.add(embeddings)

# Query vector
query = np.array([[10.0, 11.0, 12.0]])
D, I = index.search(query, 2)

for i in range(D.shape[0]):
    print(f"Distance: {D[i]}, Index: {I[i]}")


Distance: [ 27. 108.], Index: [2 1]


In [19]:
pip install openai

In [ ]:
# Chat History Memory

In [21]:
class FileChatMessageHistory:
    def __init__(self, file_path):
        self.file_path = file_path

    def add_message(self, sender, message):
        with open(self.file_path, "a") as file:
            file.write(f"{sender}: {message}\n")

    def get_chat_history(self):
        with open(self.file_path, "r") as file:
            return file.readlines()

class ChatOpenAI:
    def __init__(self, temperature):
        self.temperature = temperature

    # Add any other methods or logic you need

class AutoGPT:
    def __init__(self, ai_name, ai_role, tools, llm, memory, chat_history_memory):
        self.ai_name = ai_name
        self.ai_role = ai_role
        self.tools = tools
        self.llm = llm
        self.memory = memory
        self.chat_history_memory = chat_history_memory

    @classmethod
    def from_llm_and_tools(cls, ai_name, ai_role, tools, llm, memory, chat_history_memory):
        return cls(ai_name, ai_role, tools, llm, memory, chat_history_memory)


file_chat_history = FileChatMessageHistory("chat_history.txt")


chat_open_ai_instance = ChatOpenAI(temperature=0)


memory_instance = InMemoryDocstore

agent = AutoGPT(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools, 
    llm=chat_open_ai_instance,  
    memory=memory_instance,  
    chat_history_memory=file_chat_history,
)


In [22]:

file_chat_history.add_message(sender="User", message="Hello, how are you?")

In [23]:
chat_history = file_chat_history.get_chat_history()
print("Chat History:")
print(chat_history)

Chat History:
['User: Hello, how are you?\n', 'User: Hello, how are you?\n']
